## croplearn.ipynb

In [1]:
import sys
sys.path.insert(0,'../cropseg/')

datasetinfo = { "datadir":"/home/ucfaab0/Desktop/su_african_crops_ghana/",
                "metadatadir":"/home/ucfaab0/Desktop/su_african_crops_ghana/metadata/",
                "dataset":"su_african_crops_ghana",
                "groundcollection":"su_african_crops_ghana_labels",
                "s1collection":"su_african_crops_ghana_source_s1",
                "s2collection":"su_african_crops_ghana_source_s2",
                "groundlabels":"su_african_crops_ghana_labels_id.json",
                "groundmetadata":"su_african_crops_ghana_labels.json",
                "s1metadata":"su_african_crops_ghana_source_s1.json",
                "s2metadata":"su_african_crops_ghana_source_s2.json",
                "groundname":"labels.tif",
                "s1imagename":"source.tif",
                "s2imagename":"source.tif",
                "s2maskname":"cloudmask.tif",
                "groundshape":[64,64],
                "s1shape":[64,64],
                "s2shape":[64,64],
                "extension":"tif"
              }
s1bands = [
            {"band":"vv","idx":0},
            {"band":"vh","idx":1},    
          ]  
s2bands = [
            {"band":"blue","wavelength":490,"idx":0},
            {"band":"green","wavelength":560,"idx":1},
            {"band":"red","wavelength":665,"idx":2},
            {"band":"rded1","wavelength":705,"idx":3},
            {"band":"rded2","wavelength":740,"idx":4},
            {"band":"rded3","wavelength":783,"idx":5},
            {"band":"nir","wavelength":842,"idx":6},
            {"band":"rded4","wavelength":865,"idx":7},
            {"band":"swir1","wavelength":1610,"idx":8},
            {"band":"swir2","wavelength":2190,"idx":9}
          ]
s1indices = ["vhvv"]
s2indices = ["ndvi","rdedci","ndmi"]

from mlhubdata import loadjson
groundlabels = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["groundlabels"]}')
groundmetadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["groundmetadata"]}')
s1metadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["s1metadata"]}')
s2metadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["s2metadata"]}')

skiplist = ["001268"]

In [2]:
#####
erosioniterations = 0
vhvv = 2
ndvi = 10
ncoeff = 15
#####

import numpy
from osgeo import gdal
import scipy.interpolate

from grounddata import erodedfieldmasks
from mlhubdata import get_tileitems_from_collection
from satellitedata import load_satellite_data_as_array
from satellitedata import load_satellite_cloudmasks_as_array
from compression import dct
from compression import dct_fittingconditions
from compression import doublelogistic
from compression import doublelogistic_fittingconditions
from compression import doublelogistic_parameterconditions
from dates import datepositions

data = []
for i in range(len(groundmetadata)):
    print(i+1,"/",len(groundmetadata),end="\r")
    tileid = groundmetadata[i]["id"].split("_")[len(groundmetadata[i]["id"].split("_"))-1]
    if tileid not in skiplist:
        tilehandle = gdal.Open(f'{datasetinfo["datadir"]}{datasetinfo["groundcollection"]}/{datasetinfo["groundcollection"]}_{tileid}/{datasetinfo["groundname"]}')
        tiledata = numpy.array(tilehandle.GetRasterBand(1).ReadAsArray(),dtype="int")
        crops = numpy.unique(tiledata[tiledata != 0])
        fieldmasks = erodedfieldmasks(tiledata,erosioniterations)    
        s1items,s1dates = get_tileitems_from_collection(tileid,s1metadata,datasetinfo,verbose=0)
        s1data = load_satellite_data_as_array(s1items,s1bands,s1indices,datasetinfo,datasetinfo["s1shape"])
        s2items,s2dates = get_tileitems_from_collection(tileid,s2metadata,datasetinfo,verbose=0)
        s2data = load_satellite_data_as_array(s2items,s2bands,s2indices,datasetinfo,datasetinfo["s2shape"],rr=4096.)
        s2cloudmasks = load_satellite_cloudmasks_as_array(s2items,datasetinfo,datasetinfo["s2shape"])
        for j in range(len(fieldmasks)):
            for k in range(datasetinfo["groundshape"][0]):
                for m in range(datasetinfo["groundshape"][0]):
                    if fieldmasks[j][0][k][m] != 0:
                        s1 = False
                        s2 = False
                        tmp_s1data = []
                        tmp_s1dates = []                        
                        tmp_s2data = []
                        tmp_s2dates = []
                        for n in range(len(s1items)):
                            tmp_s1data.append(s1data[n][vhvv][k][m])
                            tmp_s1dates.append(s1dates[n])
                        if len(tmp_s1data) > 0:
                            tmp_s1data = numpy.array(tmp_s1data)
                            tmp_s1dpos = numpy.array(datepositions(tmp_s1dates)) 
                            if numpy.isnan(numpy.sum(tmp_s1data)) == False and numpy.min(tmp_s1data) > -15.0 and numpy.max(tmp_s1data) < 0.0:
                                if dct_fittingconditions(tmp_s1dpos,tmp_s1data,minduration=0.9,maxgap=0.2) == True:
                                    dctcoeff = dct(tmp_s1data,ncoeff)
                                    s1 = True   
                        for n in range(len(s2items)):
                            if s2cloudmasks[n][k][m] == 0:
                                tmp_s2data.append(s2data[n][ndvi][k][m])
                                tmp_s2dates.append(s2dates[n])
                        if len(tmp_s2data) > 0:
                            tmp_s2data = numpy.array(tmp_s2data)
                            tmp_s2dpos = numpy.array(datepositions(tmp_s2dates))   
                            if numpy.isnan(numpy.sum(tmp_s2data)) == False and doublelogistic_fittingconditions(tmp_s2dpos,tmp_s2data,durationmin=0.9,gapmax=0.5,amplitudemin=0.25) == True:
                                tmp_s2dposnew = numpy.linspace(numpy.min(tmp_s2dpos),numpy.max(tmp_s2dpos),1000)
                                smoothedspline = scipy.interpolate.UnivariateSpline(tmp_s2dpos,tmp_s2data,s=0.0075,k=2)
                                dbllog = doublelogistic(tmp_s2dposnew,smoothedspline(tmp_s2dposnew),bound=True,epsilon=0.1)
                                if doublelogistic_parameterconditions(dbllog,A1min=0.05,A1max=0.25,A2min=0.30,A2max=0.80,x01min=0.50,x01max=0.70,k1min=5.0,k1max=50.0,x02min=0.80,x02max=0.95,k2min=-50.0,k2max=-10.0) == True:
                                    s2 = True
                        if s1 == True and s2 == True:        
                            tmp = []
                            tmp.append(crops[j])
                            for n in range(len(dbllog)):
                                tmp.append(dbllog[n])
                            for n in range(len(dctcoeff)):
                                tmp.append(dctcoeff[n])
                            data.append(tmp)
data = numpy.array(data)

../cropseg/satellitedata.py:17: RuntimeWarning: invalid value encountered in true_divide
  return (data[nir] - data[red]) / (data[nir] + data[red])
../cropseg/satellitedata.py:29: RuntimeWarning: invalid value encountered in true_divide
  return (data[nir] / data[reded1]) - 1
../cropseg/satellitedata.py:33: RuntimeWarning: invalid value encountered in true_divide
  return (data[nir] - data[swir1]) / (data[nir] + data[swir1])


../cropseg/satellitedata.py:29: RuntimeWarning: divide by zero encountered in true_divide
  return (data[nir] / data[reded1]) - 1


../cropseg/compression.py:15: RuntimeWarning: overflow encountered in exp
  return A1+A2*(1./(1+numpy.exp(-k1*(x-x01)))-1./(1+numpy.exp(k2*(x-x02))))


In [20]:
d = numpy.copy(data)
numpy.random.shuffle(d)
d[:,0] = d[:,0] - 1
d0 = d[d[:,0] == 0][:10965]
d1 = d[d[:,0] == 1][:10965]
d2 = d[d[:,0] == 2][:10965]
d3 = d[d[:,0] == 3][:10965]
d = numpy.concatenate([d0,d1,d2,d3])
numpy.random.shuffle(d)
xtrain, ytrain = d[1000:,1:],d[1000:,0] 
xtest, ytest = d[:1000,1:],d[:1000,0]

In [32]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
model = tf.keras.Sequential([
    layers.Dense(21*2,activation='relu'),
    layers.Dense(21*3,activation='relu'),
    layers.Dense(21*4,activation='relu'),
    layers.Dense(4)
])
model.compile(optimizer='Adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=10)
model.summary()

Epoch 1/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.3806 - accuracy: 0.3486
Epoch 2/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.3067 - accuracy: 0.3792
Epoch 3/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2984 - accuracy: 0.3866
Epoch 4/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2915 - accuracy: 0.3973
Epoch 5/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2878 - accuracy: 0.3942
Epoch 6/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2830 - accuracy: 0.3981
Epoch 7/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2798 - accuracy: 0.4031
Epoch 8/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2725 - accuracy: 0.4061
Epoch 9/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.2699 - accuracy: 0.4070
Epoch 10/10
1340/1340 [==============================] - 2s 1ms/step - loss: 1.263

In [33]:
import sklearn.metrics
model.evaluate(xtest,ytest,verbose=1)
predictions = model.predict(xtest)
predict = numpy.argmax(predictions,axis = 1)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

32/32 [==============================] - 0s 814us/step - loss: 1.2535 - accuracy: 0.4290
[0.45945946 0.29561201 0.51072961 0.43222004] 0.42682235360976023
0.23668714198822283


In [34]:
import sklearn.dummy
dummy = sklearn.dummy.DummyClassifier(strategy="uniform")
dummy.fit(xtrain,ytrain)
predict = dummy.predict(xtest)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.accuracy_score(true,predict))
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

0.253
[0.25       0.23505976 0.26746507 0.25963489] 0.25313069397921384
0.004806769978470737
